In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import time
import os
import copy
from torch.utils.data import random_split, DataLoader
# Set the path for the dataset
data_dir = r"C:\Users\venug\Downloads\archive\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\Reduced_Dataset"
classes = ['Cyst', 'Normal', 'Stone', 'Tumor']


In [8]:
# Define data transforms
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load all images from the dataset directory
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms)

# Split into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size   # 20% for validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
class_names = full_dataset.classes

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [9]:
# Function to train and evaluate the model
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it has better accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [10]:
# Function to set up and fine-tune a model
def fine_tune_model(architecture_name, num_classes=4):
    if architecture_name == 'resnet18':
        model = models.resnet18(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)

    elif architecture_name == 'vgg16':
        model = models.vgg16(pretrained=True)
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, num_classes)

    elif architecture_name == 'mobilenet_v2':
        model = models.mobilenet_v2(pretrained=True)
        num_ftrs = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, num_classes)

    else:
        raise ValueError("Unknown architecture. Choose from 'resnet18', 'vgg16', or 'mobilenet_v2'")

    model = model.to(device)

    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    return model, criterion, optimizer, exp_lr_scheduler


In [10]:
arch = 'resnet18'
print(f"Fine-tuning {arch}...")
model, criterion, optimizer, scheduler = fine_tune_model(arch)
model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

    # Save the model weights
torch.save(model.state_dict(), f'{arch}_fine_tuned.pth')
print(f"Model {arch} saved!\n")

Fine-tuning resnet18...
Epoch 0/9
----------
train Loss: 0.7040 Acc: 0.7516
val Loss: 0.2962 Acc: 0.9068

Epoch 1/9
----------
train Loss: 0.2024 Acc: 0.9449
val Loss: 0.1340 Acc: 0.9678

Epoch 2/9
----------
train Loss: 0.0804 Acc: 0.9843
val Loss: 0.0632 Acc: 0.9871

Epoch 3/9
----------
train Loss: 0.0479 Acc: 0.9928
val Loss: 0.0479 Acc: 0.9904

Epoch 4/9
----------
train Loss: 0.0291 Acc: 0.9960
val Loss: 0.0319 Acc: 0.9936

Epoch 5/9
----------
train Loss: 0.0207 Acc: 0.9980
val Loss: 0.0237 Acc: 0.9952

Epoch 6/9
----------
train Loss: 0.0139 Acc: 0.9988
val Loss: 0.0233 Acc: 0.9952

Epoch 7/9
----------
train Loss: 0.0149 Acc: 0.9992
val Loss: 0.0246 Acc: 0.9952

Epoch 8/9
----------
train Loss: 0.0105 Acc: 0.9996
val Loss: 0.0217 Acc: 0.9952

Epoch 9/9
----------
train Loss: 0.0114 Acc: 1.0000
val Loss: 0.0277 Acc: 0.9920

Training complete in 20m 44s
Best val Acc: 0.9952
Model resnet18 saved!

Fine-tuning vgg16...


c:\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\venug/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [02:26<00:00, 3.77MB/s] 


Epoch 0/9
----------


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
